# Example Seldon Core Deployments using Ksonnet


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

** If running on GCP then run following to add cluster-admin to your user account **

In [ ]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")

On most clusters you will probably need to add cluster-admin privledges

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [ ]:
!rm -rf my-ml-deployment && ks init my-ml-deployment 

In [ ]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=true --withAmbassador=true --singleNamespace=false --namespace=seldon --withRbac=true

In [ ]:
!cd my-ml-deployment && \
      ks apply default

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

## Create Model App

In [ ]:
!kubectl create namespace test1

In [ ]:
!rm -rf my-model && ks init my-model --namespace test1

In [ ]:
!cd my-model && \
    ks registry add seldon-core ../../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master

## Serve Single Model

In [ ]:
!cd my-model && \
    ks generate seldon-serve-simple-v1alpha2 mymodel --image seldonio/mock_classifier:1.0 --oauthKey=key --oauthSecret=secret && \
    ks apply default -c mymodel

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-025d03d -n test1

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="test1",oauth_key="key",oauth_secret="secret")

#### REST Request

In [ ]:
p = sc.predict(gateway="seldon",transport="rest")
print(p)

In [ ]:
p = sc.predict(gateway="ambassador",transport="rest")
print(p)

#### gRPC Request

In [ ]:
p = sc.predict(gateway="ambassador",transport="grpc")
print(p)

In [ ]:
p = sc.predict(gateway="seldon",transport="grpc")
print(p)

In [ ]:
!cd my-model && \
    ks delete default -c mymodel && \
    ks component rm mymodel

## Serve AB Test

In [ ]:
!cd my-model && \
    ks generate seldon-abtest-v1alpha2 myabtest --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 --oauthKey=key --oauthSecret=secret && \
    ks apply default -c myabtest

In [ ]:
!kubectl rollout status deploy/myabtest-myabtest-41de5b8 -n test1
!kubectl rollout status deploy/myabtest-myabtest-df66c5c -n test1

### Get predictions

In [ ]:
sc = SeldonClient(deployment_name="myabtest",namespace="test1",oauth_key="key",oauth_secret="secret")

#### REST Request

In [ ]:
r = sc.predict(gateway="seldon",transport="rest")
print(r)

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

In [ ]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

In [ ]:
!cd my-model && \
    ks delete default -c myabtest && \
    ks component rm myabtest

## Serve Multi-Armed Bandit

In [ ]:
!cd my-model && \
    ks generate seldon-mab-v1alpha2 mymab --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 --oauthKey=key --oauthSecret=secret && \
    ks apply default -c mymab

In [ ]:
!kubectl rollout status deploy/mymab-mymab-41de5b8 -n test1
!kubectl rollout status deploy/mymab-mymab-b8038b2 -n test1
!kubectl rollout status deploy/mymab-mymab-df66c5c -n test1

### Get predictions

In [ ]:
sc = SeldonClient(deployment_name="mymab",namespace="test1",oauth_key="key",oauth_secret="secret")

#### REST Request

In [ ]:
r = sc.predict(gateway="seldon",transport="rest")
print(r)

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

In [ ]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

In [ ]:
!cd my-model && \
    ks delete default -c mymab && \
    ks component rm mymab

# Tear down

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment